### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic
from collections import Counter

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [18]:
# %%pycodestyle


class MyDecisionTreeClassifier():
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None

        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise ValueError(criterion)

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise ValueError(max_features)

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_c / (l_c + r_c) * (1 - np.dot(l_s, l_s)) + \
            r_c / (l_c + r_c) * (1 - np.dot(r_s, r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_c / (l_c + r_c) * (1 - np.max(l_s)) + \
            r_c / (l_c + r_c) * (1 - np.max(r_s))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return - l_c / (l_c + r_c) * np.sum(l_s * np.log(l_s)) - \
            r_c / (l_c + r_c) * np.sum(r_s * np.log(r_s))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        min_err = None

        for feature_id in self.get_feature_ids(x.shape[1]):
            if min_err == 0:
                break
            for x_threshold in np.unique(x[:, feature_id]):
                x_left, x_right, y_left, y_right = self.__div_samples(
                    x, y, feature_id, x_threshold)

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                left_count = np.unique(
                    y_left, return_counts=True)[1] / len(y_left)
                right_count = np.unique(
                    y_right, return_counts=True)[1] / len(y_right)

                err = self.G_function(
                    len(y_left), left_count, len(y_right), right_count)
                if min_err is None or err < min_err:
                    min_err = err
                    optimal_feature_id = feature_id
                    optimal_threshold = x_threshold
                if min_err == 0:
                    break

        if min_err is None:
            return self.__find_threshold(x, y)

        y_freq = np.unique(y, return_counts=True)[1] / len(y)
        if self.G_function == self.__gini:
            impurity = 1 - np.dot(y_freq, y_freq)
        elif self.G_function == self.__misclass:
            impurity = 1 - np.max(y_freq)
        elif self.G_function == self.__entropy:
            impurity = -np.sum(y_freq * np.log(y_freq))

        self.feature_importances_[optimal_feature_id] += impurity - min_err

        return optimal_feature_id, optimal_threshold

    def __fit_node(self, x, y, node_id, depth):

        if node_id == 0:
            self.feature_importances_ = np.zeros(x.shape[1])
            self.classes_ = np.unique(y)
        if len(y) == 0:
            return

        if depth == self.max_depth or len(y) < \
                self.min_samples_split or np.all(y == y[0]):
            count = Counter(y)
            y_proba = [count[class_] / len(y) for class_ in self.classes_]
            y_pred = count.most_common(1)[0][0]
            self.tree[node_id] = (self.__class__.LEAF_TYPE, y_pred, y_proba)
        else:
            feature_id, threshold = self.__find_threshold(x, y)
            self.tree[node_id] = (
                self.__class__.NON_LEAF_TYPE,
                feature_id,
                threshold)
            x_left, x_right, y_left, y_right = self.__div_samples(
                x, y, feature_id, threshold)
            self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)

        if node_id == 0:
            self.feature_importances_ /= self.feature_importances_.sum()

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [19]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [20]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [21]:
%time clf.fit(X_train, y_train)

Wall time: 1.98 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [22]:
%time my_clf.fit(X_train, y_train)

Wall time: 439 ms


## Проверка качества работы на wine

In [23]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8897028897028897

In [24]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

## Подготовка данных Speed Dating Data 

In [25]:
# %%pycodestyle


import pandas as pd

df = pd.read_csv('dating.csv', encoding="ISO-8859-1")

df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df = df.dropna(subset=['iid'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)

df = df.drop(['undergra'], axis=1)
df = df.drop(['mn_sat'], axis=1)
df = df.drop(['tuition'], axis=1)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df = df.drop(['income'], axis=1)

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)


df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga'], axis=1)

feat = ['iid', 'wave', 'attr1_1', 'sinc1_1',
        'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

df.loc[:, 'temp_totalsum'] = \
                    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                               'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1',
           'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100


feat = ['iid', 'wave', 'attr2_1', 'sinc2_1',
        'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1', 'amb2_1',
                                        'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)


df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match',
                                          'int_corr', 'samerace'], axis=1)\
                                   .dropna()

df_female.columns = df_female.columns + '_f'


data = pd.merge(df_male, df_female, left_on='pid', right_on='iid_f')


In [83]:
X_data = data.drop(['match', 'iid', 'pid'], axis = 1)
y_data = data['match']

In [84]:
X = X_data.values
y = y_data.values

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [109]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [110]:
%time clf.fit(X_train, y_train)

Wall time: 15.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [111]:
%time my_clf.fit(X_train, y_train)

Wall time: 2.59 s


## Проверка качества работы на Speed Dating Data

In [112]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5868222374742622

In [113]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6261725005719515

## Задание 3

In [114]:
# %%pycodestyle

ind = np.argsort(clf.feature_importances_)[::-1][:10]
clf_answer = pd.Series(clf.feature_importances_[ind], X_data.columns[ind])

ind = np.argsort(my_clf.feature_importances_)[::-1][:10]
my_clf_answer = pd.Series(my_clf.feature_importances_[ind],
                          X_data.columns[ind])


## Задание 4

Результат по DecisionTreeClassifier:

In [115]:
clf_answer

int_corr      0.121816
attr3_1       0.081564
shar1_1       0.072865
imprelig_f    0.047614
imprace       0.031858
age           0.031485
intel3_1_f    0.029905
iid_f         0.028053
attr2_1       0.025822
attr1_1       0.025751
dtype: float64

Результат по MyDecisionTreeClassifier:

In [116]:
my_clf_answer

int_corr      0.171102
sinc1_1       0.063302
intel2_1      0.052023
imprace_f     0.046120
sinc1_1_f     0.043997
sinc2_1       0.038498
shar2_1_f     0.031805
expnum_f      0.028735
field_cd_f    0.028295
intel3_1      0.027656
dtype: float64

## Задание 5

In [117]:
# %%pycodestyle

from scipy.stats import randint
cv = KFold(n_splits=10, random_state=123, shuffle=True)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': randint(2, 8)}


In [118]:
# %%pycodestyle

forest = RandomForestClassifier(n_estimators=100)

random_search = \
    RandomizedSearchCV(forest,
                       param_distributions=param_grid,
                       n_iter=30, n_jobs=-1,
                       cv=cv, scoring='roc_auc')


In [119]:
random_search.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=123, shuffle=True),
                   error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_...
                                                    random_state=None,
   

Самые удачные параметры:

In [120]:
random_search.best_params_

{'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt'}

In [121]:
random_search.best_score_

0.775918718222042

In [122]:
best_model = random_search.best_estimator_
best_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=5, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)